In [416]:
# import libraries
#import boto3, re, sys, math, json, os, sagemaker, 
import urllib.request
import numpy as np                                
import pandas as pd  
import matplotlib
import matplotlib.pyplot as plt  
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime       
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
#from sagemaker.predictor import csv_serializer  
#from sagemaker import get_execution_role

# Global variables
Data_Download_Completed = True
Use_Small_DataSet = False
Small_Data_Already_Split = True
Dominate_Component_Number = 10
Feature_Weight_Step_Threshold = 0.01


In [417]:
# Grab the necessary dataset files from the origin sources
data_dir="/home/will/Desktop/CSC8515/"
unsw_nb15_mainurl = 'https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/'
unsw_nb15_training_file = 'UNSW_NB15_training-set.csv'
unsw_nb15_feature_file = 'NUSW-NB15_features.csv' # misspelling in the origin data file name
unsw_nb15_test_file = 'UNSW_NB15_testing-set.csv'
download_training_url = unsw_nb15_mainurl + unsw_nb15_training_file
download_feature_url = unsw_nb15_mainurl + unsw_nb15_feature_file
download_test_url = unsw_nb15_mainurl + unsw_nb15_test_file

if(Data_Download_Completed !=True):
    try:
      urllib.request.urlretrieve (download_feature_url, unsw_nb15_feature_file)
      print('Success: ' + unsw_nb15_feature_file)
    except Exception as e:
      print('Data load error: ',e)

    try:
      urllib.request.urlretrieve (download_training_url, unsw_nb15_training_file)
      print('Success: ' + unsw_nb15_training_file)
    except Exception as e:
      print('Data load error: ',e)


In [418]:
# Load data
# Must declare data_dir as the directory of training and test files
raw_data_filename = data_dir + unsw_nb15_training_file
if(Use_Small_DataSet !=True):
    print('Loading raw data')
    try:
        raw_data = pd.read_csv(raw_data_filename, index_col=0, header=0)
        print('Success: Data loaded into dataframe.')
    except Exception as e:
        print('Data load error: ',e)
    selected_data = raw_data
    print(selected_data.shape)


Loading raw data
Success: Data loaded into dataframe.
(82332, 44)


In [419]:

# Split smaller dataset for experimentation
samll_filename = data_dir + 'UNSW_NB15_training-small.csv'
if((Use_Small_DataSet ==True) and (Small_Data_Already_Split !=True)):
    remaining_data, selected_data = np.split(raw_data.sample(frac=1, random_state=1729), [int(0.80 * len(raw_data))])
    print(remaining_data.shape, selected_data.shape)
    selected_data.head(10)
    # Write the samll data set to disk
    selected_data.to_csv(samll_filename)

In [420]:
# Read back the small data
if(Use_Small_DataSet ==True):
    try:
        small_data = pd.read_csv(samll_filename, index_col=0, header=0)
        print('Success: Data loaded into dataframe.')
    except Exception as e:
        print('Data load error: ',e)
    selected_data = small_data
    print(selected_data.shape)


In [421]:
    selected_data.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,Normal,0
3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,Normal,0
4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,Normal,0
5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,Normal,0


In [422]:
# Now gather the list of feature names
print('Loading feature list')
try:
    feature_data = pd.read_csv(raw_data_filename, index_col=0, header=None, nrows=1)
    print('Success: Feature list loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
#print(feature_data)
feature_list = feature_data.values[0]
feature_only_list = np.delete(feature_list, [(feature_list.shape[0]-2), (feature_list.shape[0]-1)])
print(feature_only_list.shape)
feature_only_list

Loading feature list
Success: Feature list loaded into dataframe.
(42,)


array(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss',
       'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb',
       'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean',
       'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl',
       'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
       'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports'], dtype=object)

In [427]:
# pd.crosstab(selected_data['attack_cat'], selected_data['service'], margins=True)

In [428]:
print('Transforming data')
# Factorize columns: "proto", "service", "state", "attack_cat"
selected_data['proto'], protocols = pd.factorize(selected_data['proto'])
selected_data['service'], services = pd.factorize(selected_data['service'])
selected_data['state'], states    = pd.factorize(selected_data['state'])
#selected_data['attack_cat'], attacks = pd.factorize(selected_data['attack_cat'])
selected_data.head()

Transforming data


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,0.000011,0,0,0,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000008,0,0,0,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,Normal,0
3,0.000005,0,0,0,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,Normal,0
4,0.000006,0,0,0,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,Normal,0
5,0.000010,0,0,0,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,Normal,0


In [429]:
#pd.crosstab(selected_data['attack_cat'], selected_data['service'], margins=True)

In [430]:
import matplotlib.pyplot as plt
%matplotlib inline
#selected_data.boxplot(column='service', by='attack_cat')

In [431]:
selected_data.describe()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
count,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,8.233200e+04,8.233200e+04,8.233200e+04,82332.000000,...,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000
mean,1.006756,8.811216,1.901739,0.862046,18.666472,17.545936,7.993908e+03,1.323379e+04,8.241089e+04,180.967667,...,4.928898,3.663011,7.456360,0.008284,0.008381,0.129743,6.468360,9.164262,0.011126,0.550600
std,4.710444,25.348181,2.642982,1.087298,133.916353,115.574086,1.716423e+05,1.514715e+05,1.486204e+05,101.513358,...,8.389545,5.915386,11.415191,0.091171,0.092485,0.638683,8.543927,11.121413,0.104891,0.497436
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000e+00,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000008,0.000000,0.000000,0.000000,2.000000,0.000000,1.140000e+02,0.000000e+00,2.860611e+01,62.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000
50%,0.014138,2.000000,0.000000,1.000000,6.000000,2.000000,5.340000e+02,1.780000e+02,2.650177e+03,254.000000,...,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,3.000000,5.000000,0.000000,1.000000
75%,0.719360,2.000000,6.000000,1.000000,12.000000,10.000000,1.280000e+03,9.560000e+02,1.111111e+05,254.000000,...,4.000000,3.000000,6.000000,0.000000,0.000000,0.000000,7.000000,11.000000,0.000000,1.000000
max,59.999989,130.000000,12.000000,6.000000,10646.000000,11018.000000,1.435577e+07,1.465753e+07,1.000000e+06,255.000000,...,59.000000,38.000000,63.000000,2.000000,2.000000,16.000000,60.000000,62.000000,1.000000,1.000000


In [434]:
# separate features (columns 1..42) and labels (column 43, 44)
features= pd.DataFrame(selected_data.iloc[:,:selected_data.shape[1]-2])
#labels= selected_data.iloc[:,selected_data.shape[1]-1:]
attack_labels=pd.DataFrame(selected_data['attack_cat'])
print('features shape is:', features.shape, 'labels shape is:', attack_labels.shape)
attack_labels.head(10)
df_labels= pd.DataFrame(selected_data.iloc[:,selected_data.shape[1]-2:])


features shape is: (82332, 42) labels shape is: (82332, 1)


In [444]:
# Study the training data
labels_array= attack_labels.values # this becomes a 'horizontal' array
#print(labels)
UNSW_NB15_Label_names2 = np.unique(labels_array, return_counts=True)
#print(UNSW_NB15_Label_names2)
UNSW_NB15_Label_names = {}
for i in range(len(UNSW_NB15_Label_names2[0])):
    UNSW_NB15_Label_names[UNSW_NB15_Label_names2[0][i]] = UNSW_NB15_Label_names2[1][i]
# ordered_UNSW_NB15_Label_names = sorted(UNSW_NB15_Label_names.items(), key=lambda x: x[1], reverse=True)
ordered_UNSW_NB15_Label_names = sorted(UNSW_NB15_Label_names, key=UNSW_NB15_Label_names.__getitem__, reverse=True)
print(ordered_UNSW_NB15_Label_names)
#UNSW_NB15_Label_names = np.sort(UNSW_NB15_Label_names, axis=0, order=Count)
print('Attack Category', '\t\t\t Counts')
short_cat_name = np.copy(ordered_UNSW_NB15_Label_names)
for i, cat in enumerate(short_cat_name):
    if(cat == 'Reconnaissance'):
        short_cat_name[i] = 'Reconn'
for i, cat in enumerate(short_cat_name):
    if(len(cat) < 6):
        print('\t', cat, '\t\t\t\t', UNSW_NB15_Label_names[ordered_UNSW_NB15_Label_names[i]])
    else:
        print('\t', cat, '\t\t\t', UNSW_NB15_Label_names[ordered_UNSW_NB15_Label_names[i]])
print('Total # of Data Points: \t\t', attack_labels.shape[0])

    

['Normal', 'Generic', 'Exploits', 'Fuzzers', 'DoS', 'Reconnaissance', 'Analysis', 'Backdoor', 'Shellcode', 'Worms']
Attack Category 			 Counts
	 Normal 			 37000
	 Generic 			 18871
	 Exploits 			 11132
	 Fuzzers 			 6062
	 DoS 				 4089
	 Reconn 			 3496
	 Analysis 			 677
	 Backdoor 			 583
	 Shellcode 			 378
	 Worms 				 44
Total # of Data Points: 		 82332


In [436]:
# Univariate Selection requries non-negative data
#normalized_features = features.sub(features.mean(axis=0), axis=1)
normalized_features_univ = features
normalized_features_univ = normalized_features_univ.add(normalized_features_univ.min(axis=0), axis=1)
normalized_features_univ = normalized_features_univ.divide(normalized_features_univ.max(axis=0), axis=1)
normalized_features_univ.describe()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
count,8.233200e+04,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,...,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000
mean,1.677927e-02,0.067779,0.158478,0.143674,0.001847,0.001592,0.000559,0.000903,0.082411,0.709677,...,0.112415,0.098815,0.119564,0.132131,0.004142,0.004190,0.008109,0.122432,0.161337,0.011126
std,7.850742e-02,0.194986,0.220248,0.181216,0.012578,0.010490,0.011956,0.010334,0.148620,0.398092,...,0.140302,0.139826,0.151677,0.178362,0.045586,0.046243,0.039918,0.140064,0.176530,0.104891
min,0.000000e+00,0.000000,0.000000,0.000000,0.000188,0.000000,0.000003,0.000000,0.000000,0.000000,...,0.033333,0.033333,0.051282,0.031250,0.000000,0.000000,0.000000,0.032787,0.031746,0.000000
25%,1.333334e-07,0.000000,0.000000,0.000000,0.000282,0.000000,0.000010,0.000000,0.000029,0.243137,...,0.033333,0.033333,0.051282,0.031250,0.000000,0.000000,0.000000,0.032787,0.047619,0.000000
50%,2.356334e-04,0.015385,0.000000,0.166667,0.000657,0.000182,0.000039,0.000012,0.002650,0.996078,...,0.050000,0.033333,0.051282,0.062500,0.000000,0.000000,0.000000,0.065574,0.095238,0.000000
75%,1.198934e-02,0.015385,0.500000,0.166667,0.001221,0.000908,0.000091,0.000065,0.111111,0.996078,...,0.116667,0.083333,0.102564,0.109375,0.000000,0.000000,0.000000,0.131148,0.190476,0.000000
max,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [437]:
normalized_data_univ = normalized_features_univ.merge(df_labels, how='inner', left_on='id', right_on='id')
normalized_data_univ.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,1.833334e-07,0.0,0.0,0.0,0.000282,0.0,0.000036,0.0,0.090909,0.996078,...,0.051282,0.046875,0.0,0.0,0.0,0.032787,0.047619,0.0,Normal,0
2,1.333334e-07,0.0,0.0,0.0,0.000282,0.0,0.000124,0.0,0.125000,0.996078,...,0.051282,0.046875,0.0,0.0,0.0,0.032787,0.047619,0.0,Normal,0
3,8.333335e-08,0.0,0.0,0.0,0.000282,0.0,0.000076,0.0,0.200000,0.996078,...,0.051282,0.062500,0.0,0.0,0.0,0.032787,0.063492,0.0,Normal,0
4,1.000000e-07,0.0,0.0,0.0,0.000282,0.0,0.000064,0.0,0.166667,0.996078,...,0.051282,0.062500,0.0,0.0,0.0,0.049180,0.063492,0.0,Normal,0
5,1.666667e-07,0.0,0.0,0.0,0.000282,0.0,0.000150,0.0,0.100000,0.996078,...,0.051282,0.062500,0.0,0.0,0.0,0.049180,0.063492,0.0,Normal,0


In [439]:
# normalized_data_univ.hist(column='service', by='attack_cat')

In [440]:
# This function will add up composite weight for all original features: sum(PCA_component * component weight)
# Then sort and return all features that have a big enough step threshold compared to the next smaller weight 
def get_important_features(weighted_comp, Comp_Num, feature_name_list):
    weighted_features = {}
    top_sorted_feature_weight = {}
    for i in range(len(feature_name_list)):
        weighted_features[feature_name_list[i]] = weighted_comp[i]
        
    sorted_features = sorted(weighted_features, key=weighted_features.__getitem__, reverse=True)
    
    for k in sorted_features[0:Comp_Num]:
        last_weight = 1
        if(weighted_features[k] > (Feature_Weight_Step_Threshold*last_weight)):
            # print("{} : {}".format(k, weighted_features[k]))
            top_sorted_feature_weight[k] = weighted_features[k]
            last_weight = weighted_features[k]
    return(top_sorted_feature_weight)


In [441]:
# This function takes a dataset and use PCA to find the most dominant features
def PCA_feature_finder(features, labels, feature_name_list):
    # set up a PCA learner
    pca = PCA(n_components = Dominate_Component_Number)
    # actually run the fit algorithm
    fit = pca.fit(features)
    # transform our data using the learned transform
    reduced_feature_data = fit.transform(features)
    
    weighted_comp = []
    for i in range(len(feature_name_list)):
        weighted_comp.append(0)
    for i, weight in enumerate(pca.explained_variance_ratio_):
        weighted_comp += pca.components_[i] * weight
        
    return(get_important_features(weighted_comp, Dominate_Component_Number, feature_name_list))

In [442]:
# This function takes a dataset and use univariate statistical tests to find the most dominant features
# Chi Square Test is used in statistics to test the independence of two events.
def UniV_feature_finder(features, lables, feature_name_list):
    # set up a univerate learner
    univ = SelectKBest(score_func=chi2, k=Dominate_Component_Number)
    # actually run the fit algorithm
    fit = univ.fit(features, lables)
    # transform our data using the learned transform
    reduced_feature_data = fit.transform(features)
    
        
    return(get_important_features(fit.scores_, Dominate_Component_Number, feature_name_list))


In [449]:
# This will calculate the combined univariate selection results
print("Univariate Feature selction results:")
univ_features = {}
for val in ordered_UNSW_NB15_Label_names:
    if (val != 'Normal'):
        binary_criteria = ['Normal', val]
        print('\nFor attack_cat of', val, 'the dominant features are:')
        binary_data = normalized_data_univ[normalized_data_univ.attack_cat.isin(binary_criteria)]
        # binary_data.head(10)

        # separate features (columns 1..42) and labels (column 43, 44)
        binary_features= binary_data.iloc[:,:binary_data.shape[1]-2]
        binary_attack_labels=binary_data['attack_cat']
        # print('features shape is:', binary_features.shape, 'labels shape is:', binary_attack_labels.shape)

        # print("PCA Feature selction results:")
        # print(PCA_feature_finder(binary_features, binary_attack_labels, feature_only_list))


        univ_features.update(UniV_feature_finder(binary_features, binary_attack_labels, feature_only_list))
        # print("Univariate Feature selction results:")
        print(UniV_feature_finder(binary_features, binary_attack_labels, feature_only_list))
        
sorted_univ_features = sorted(univ_features, key=univ_features.__getitem__, reverse=True)
print("\n\nCumulative Univariate Selection Results are sorted bleow: ")
print(sorted_univ_features)

Univariate Feature selction results:

For attack_cat of Generic the dominant features are:
{'swin': 12938.096142578797, 'dwin': 12090.01627589768, 'service': 10091.293712046816, 'dttl': 6847.0370974391135, 'ct_dst_sport_ltm': 6141.410097600104, 'stcpb': 6021.5869011116465, 'dtcpb': 5982.515206166755, 'ct_dst_src_ltm': 5292.06587324085, 'ct_src_dport_ltm': 5235.625993564707, 'ct_srv_dst': 4867.104349692721}

For attack_cat of Exploits the dominant features are:
{'proto': 4224.09009496089, 'dttl': 1280.6684514157303, 'dload': 644.8548009907722, 'state': 528.4159173062278, 'sttl': 340.37443685599095, 'is_sm_ips_ports': 275.59221621621623, 'sinpkt': 264.2657396487954, 'ct_srv_dst': 154.84281873737962, 'ct_state_ttl': 146.62430670940444, 'ct_srv_src': 143.3547845257794}

For attack_cat of Fuzzers the dominant features are:
{'proto': 2600.177210642371, 'sttl': 2408.295172486197, 'state': 449.9898586596669, 'dmean': 422.82864908760473, 'dload': 397.80161425726664, 'dttl': 377.57332678251197, 

In [450]:
# Normalize the dataset
# z_scaler = StandardScaler()
# normalized_features = z_scaler.fit_transform(features)
# normalized_features[0:3]

In [451]:
#normalized_features = features.sub(features.mean(axis=0), axis=1)
normalized_features_PCA = features
normalized_features_PCA = normalized_features_PCA.sub(normalized_features_PCA.mean(axis=0), axis=1)
normalized_features_PCA = normalized_features_PCA.divide(normalized_features_PCA.std(axis=0), axis=1)
normalized_features_PCA.describe()


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
count,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,...,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04,8.233200e+04
mean,3.476857e-15,7.703188e-14,-2.705665e-13,6.047640e-14,1.072367e-14,1.193902e-14,-2.574517e-15,-2.990010e-15,4.654311e-14,1.036637e-13,...,-5.034774e-14,-1.512094e-13,3.029995e-13,5.445920e-14,-2.755410e-15,-2.002353e-14,3.314717e-15,4.094806e-14,-1.086325e-14,-7.248588e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.137285e-01,-3.476074e-01,-7.195431e-01,-7.928335e-01,-1.319217e-01,-1.518155e-01,-4.643325e-02,-8.736818e-02,-5.545060e-01,-1.782698e+00,...,-5.636564e-01,-4.683088e-01,-4.501838e-01,-5.655936e-01,-9.085693e-02,-9.061681e-02,-2.031416e-01,-6.400289e-01,-7.341029e-01,-1.060694e-01
25%,-2.137268e-01,-3.476074e-01,-7.195431e-01,-7.928335e-01,-1.244543e-01,-1.518155e-01,-4.590890e-02,-8.736818e-02,-5.543135e-01,-1.171941e+00,...,-5.636564e-01,-4.683088e-01,-4.501838e-01,-5.655936e-01,-9.085693e-02,-9.061681e-02,-2.031416e-01,-6.400289e-01,-6.441863e-01,-1.060694e-01
50%,-2.107271e-01,-2.687063e-01,-7.195431e-01,1.268775e-01,-9.458496e-02,-1.345106e-01,-4.346195e-02,-8.619304e-02,-5.366742e-01,7.194357e-01,...,-4.448650e-01,-4.683088e-01,-4.501838e-01,-3.903885e-01,-9.085693e-02,-9.061681e-02,-2.031416e-01,-4.059445e-01,-3.744364e-01,-1.060694e-01
75%,-6.101248e-02,-2.687063e-01,1.550620e+00,1.268775e-01,-4.978087e-02,-6.529090e-02,-3.911571e-02,-8.105676e-02,1.931110e-01,7.194357e-01,...,3.030098e-02,-1.107209e-01,-1.120824e-01,-1.275808e-01,-9.085693e-02,-9.061681e-02,-2.031416e-01,6.222434e-02,1.650634e-01,-1.060694e-01
max,1.252392e+01,4.780966e+00,3.820783e+00,4.725432e+00,7.935800e+01,9.518097e+01,8.359119e+01,9.668024e+01,6.174047e+00,7.292866e-01,...,6.326250e+00,6.445058e+00,5.804691e+00,4.865765e+00,2.184590e+01,2.153449e+01,2.484843e+01,6.265461e+00,4.750811e+00,9.427673e+00


In [457]:
normalized_data_PCA = normalized_features_PCA.merge(df_labels, how='inner', left_on='id', right_on='id')
normalized_data_PCA.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,-0.213726,-0.347607,-0.719543,-0.792833,-0.124454,-0.151815,-0.043683,-0.087368,0.057181,0.719436,...,-0.450184,-0.477991,-0.090857,-0.090617,-0.203142,-0.640029,-0.644186,-0.106069,Normal,0
2,-0.213727,-0.347607,-0.719543,-0.792833,-0.124454,-0.151815,-0.036308,-0.087368,0.286563,0.719436,...,-0.450184,-0.477991,-0.090857,-0.090617,-0.203142,-0.640029,-0.644186,-0.106069,Normal,0
3,-0.213727,-0.347607,-0.719543,-0.792833,-0.124454,-0.151815,-0.040351,-0.087368,0.791205,0.719436,...,-0.450184,-0.390389,-0.090857,-0.090617,-0.203142,-0.640029,-0.554270,-0.106069,Normal,0
4,-0.213727,-0.347607,-0.719543,-0.792833,-0.124454,-0.151815,-0.041330,-0.087368,0.566919,0.719436,...,-0.450184,-0.390389,-0.090857,-0.090617,-0.203142,-0.522987,-0.554270,-0.106069,Normal,0
5,-0.213726,-0.347607,-0.719543,-0.792833,-0.124454,-0.151815,-0.034187,-0.087368,0.118349,0.719436,...,-0.450184,-0.390389,-0.090857,-0.090617,-0.203142,-0.522987,-0.554270,-0.106069,Normal,0


In [458]:
# This will calculate the combined univariate selection results
print("PCA Feature selction results:")
pca_features = {}
for val in ordered_UNSW_NB15_Label_names:
    if (val != 'Normal'):
        binary_criteria = ['Normal', val]
        print('\nFor attack_cat of', val, 'the dominant features are:')
        binary_data = normalized_data_PCA[normalized_data_PCA.attack_cat.isin(binary_criteria)]
        binary_data.head(10)

        # separate features (columns 1..42) and labels (column 43, 44)
        binary_features= binary_data.iloc[:,:binary_data.shape[1]-2]
        binary_attack_labels=binary_data['attack_cat']
        # print('features shape is:', binary_features.shape, 'labels shape is:', binary_attack_labels.shape)


        pca_features.update(PCA_feature_finder(binary_features, binary_attack_labels, feature_only_list))
        # print("PCA Feature selction results:")
        print(PCA_feature_finder(binary_features, binary_attack_labels, feature_only_list))

        #print("Univariate Feature selction results:")
        #print(UniV_feature_finder(binary_features, binary_attack_labels, feature_only_list))

        
sorted_pca_features = sorted(pca_features, key=pca_features.__getitem__, reverse=True)
print("\n\nCumulative Univariate Selection Results are sorted bleow: ")
print(sorted_pca_features)

PCA Feature selction results:

For attack_cat of Generic the dominant features are:
{'dwin': 0.11234833763131065, 'swin': 0.11002991784667461, 'stcpb': 0.09397486129450121, 'dtcpb': 0.09336462058759169, 'dttl': 0.08448533832147831, 'tcprtt': 0.0755812209342999, 'ackdat': 0.07181708565415147, 'synack': 0.06792054454489431, 'dmean': 0.05916734101703472, 'djit': 0.05320277399891214}

For attack_cat of Exploits the dominant features are:
{'spkts': 0.08883834215091664, 'ct_flw_http_mthd': 0.08882904600848597, 'sloss': 0.08699400639220765, 'sbytes': 0.08645968513700175, 'trans_depth': 0.08211560290847131, 'djit': 0.046961037388597304, 'dwin': 0.04438818957056326, 'swin': 0.03828769696172392, 'dtcpb': 0.037567783209654246, 'is_ftp_login': 0.037261538331276225}

For attack_cat of Fuzzers the dominant features are:
{'sjit': 0.11828569804000662, 'dinpkt': 0.1118058925369726, 'djit': 0.08880422615155649, 'tcprtt': 0.08509168805921553, 'synack': 0.08161253456808629, 'ackdat': 0.07423652604669327, 

In [459]:
sorted_pca_features

['dpkts',
 'dbytes',
 'dloss',
 'stcpb',
 'sbytes',
 'tcprtt',
 'dttl',
 'synack',
 'trans_depth',
 'ct_state_ttl',
 'ackdat',
 'dinpkt',
 'ct_ftp_cmd',
 'is_ftp_login',
 'sjit',
 'sinpkt',
 'djit',
 'ct_flw_http_mthd',
 'response_body_len',
 'dmean',
 'is_sm_ips_ports',
 'dwin',
 'sttl',
 'state',
 'spkts',
 'dur',
 'swin',
 'dtcpb',
 'sloss']

In [460]:
sorted_univ_features

['proto',
 'ct_dst_sport_ltm',
 'ct_src_dport_ltm',
 'sinpkt',
 'stcpb',
 'dtcpb',
 'is_sm_ips_ports',
 'rate',
 'dmean',
 'swin',
 'service',
 'sttl',
 'dttl',
 'ct_state_ttl',
 'ct_flw_http_mthd',
 'dload',
 'ct_srv_src',
 'state',
 'response_body_len',
 'ct_srv_dst',
 'dwin',
 'ct_dst_src_ltm']

In [461]:
# separate features (columns 1..42) and labels (column 43, 44)
features= normalized_data_PCA[sorted_pca_features]
#labels= selected_data.iloc[:,selected_data.shape[1]-1:]
attack_labels=normalized_data_PCA['attack_cat']
print('features shape is:', features.shape, 'labels shape is:', attack_labels.shape)
attack_labels.head(10)

features shape is: (82332, 29) labels shape is: (82332,)


id
1     Normal
2     Normal
3     Normal
4     Normal
5     Normal
6     Normal
7     Normal
8     Normal
9     Normal
10    Normal
Name: attack_cat, dtype: object

In [462]:
# Separate data in train set and test set
df= pd.DataFrame(features)
# create training and testing vars
# Note: train_size + test_size < 1.0 means we are subsampling
# Use small numbers for slow classifiers, as KNN, Radius, SVC,...
X_train, X_test, y_train, y_test = train_test_split(df, labels, train_size=0.8, test_size=0.2, random_state=1)
print('X_train, y_train:', X_train.shape, y_train.shape)
print('X_test, y_test:', X_test.shape, y_test.shape)

X_train, y_train: (65865, 29) (65865,)
X_test, y_test: (16467, 29) (16467,)


In [463]:
# Training, choose model by commenting/uncommenting clf=
print('Training model...')
clf= RandomForestClassifier(n_jobs=-1, random_state=3, n_estimators=102)
#, max_features=0.8, min_samples_leaf=3, n_estimators=500, min_samples_split=3, random_state=10, verbose=1)
# clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, presort=False)
# clf= svm.SVC(kernel='rbf')
#clf= neighbors.KNeighborsClassifier(n_neighbors=1, algorithm='ball_tree', metric='manhattan')

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred, ordered_UNSW_NB15_Label_names)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred,ordered_UNSW_NB15_Label_names))


Training model...
Predicting...

Confusion matrix:
 [[7150    6   28  134    5    1    0    0   13    1]
 [  21 3743   57    8   22    1    0    0    1    0]
 [ 125   12 1570   88  405   23    2    1    0    0]
 [ 515    6  150  462   85    3    0    1    9    0]
 [  37    5  260   39  467    8    0    0    3    0]
 [  27    0   68    9   52  530    0    1    0    0]
 [   0    0   53   21   40    0    6    0    0    0]
 [   1    1   69   26   19    0    0    2    0    0]
 [  31    1    9   14    2    2    0    0    7    0]
 [   4    0    4    0    1    0    0    0    0    0]]
Computing performance metrics

Accuracy result: 0.8463593854375417

Classification report:
                precision    recall  f1-score   support

        Normal       0.90      0.97      0.94      7338
       Generic       0.99      0.97      0.98      3853
      Exploits       0.69      0.71      0.70      2226
       Fuzzers       0.58      0.38      0.45      1231
           DoS       0.43      0.57      0.49

In [464]:
# Function to plot confusion matrix
# https://gist.github.com/zachguo/10296432
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [1])  # 5 is value length
    empty_cell = " " * columnwidth
    empty_cell0 = " " * (columnwidth-1) + "-"
    
    # Begin CHANGES
    fst_empty_cell = (columnwidth-3)//2 * " " + "t/p" + (columnwidth-3)//2 * " "
    
    if len(fst_empty_cell) < len(empty_cell):
        fst_empty_cell = " " * (len(empty_cell) - len(fst_empty_cell)) + fst_empty_cell
    # Print header
    print("    " + fst_empty_cell, end=" ")
    # End CHANGES
    
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
        
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell0
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()
        

In [465]:
print_cm(results, short_cat_name, True)

       t/p       Normal   Generic  Exploits   Fuzzers       DoS    Reconn  Analysis  Backdoor Shellcode     Worms 
       Normal    7150.0       6.0      28.0     134.0       5.0       1.0         -         -      13.0       1.0 
      Generic      21.0    3743.0      57.0       8.0      22.0       1.0         -         -       1.0         - 
     Exploits     125.0      12.0    1570.0      88.0     405.0      23.0       2.0       1.0         -         - 
      Fuzzers     515.0       6.0     150.0     462.0      85.0       3.0         -       1.0       9.0         - 
          DoS      37.0       5.0     260.0      39.0     467.0       8.0         -         -       3.0         - 
       Reconn      27.0         -      68.0       9.0      52.0     530.0         -       1.0         -         - 
     Analysis         -         -      53.0      21.0      40.0         -       6.0         -         -         - 
     Backdoor       1.0       1.0      69.0      26.0      19.0         -       

In [466]:
# separate features (columns 1..42) and labels (column 43, 44)
features= normalized_data_univ[sorted_univ_features]
#labels= selected_data.iloc[:,selected_data.shape[1]-1:]
attack_labels=normalized_data_PCA['attack_cat']
print('features shape is:', features.shape, 'labels shape is:', attack_labels.shape)
attack_labels.head(10)

features shape is: (82332, 22) labels shape is: (82332,)


id
1     Normal
2     Normal
3     Normal
4     Normal
5     Normal
6     Normal
7     Normal
8     Normal
9     Normal
10    Normal
Name: attack_cat, dtype: object

In [467]:
# Separate data in train set and test set
df= pd.DataFrame(features)
# create training and testing vars
# Note: train_size + test_size < 1.0 means we are subsampling
# Use small numbers for slow classifiers, as KNN, Radius, SVC,...
X_train, X_test, y_train, y_test = train_test_split(df, labels, train_size=0.8, test_size=0.2, random_state=1)
print('X_train, y_train:', X_train.shape, y_train.shape)
print('X_test, y_test:', X_test.shape, y_test.shape)

X_train, y_train: (65865, 22) (65865,)
X_test, y_test: (16467, 22) (16467,)


In [468]:
# Training, choose model by commenting/uncommenting clf=
print('Training model...')
clf= RandomForestClassifier(n_jobs=-1, random_state=3, n_estimators=102)
#, max_features=0.8, min_samples_leaf=3, n_estimators=500, min_samples_split=3, random_state=10, verbose=1)
# clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, presort=False)
# clf= svm.SVC(kernel='rbf')
#clf= neighbors.KNeighborsClassifier(n_neighbors=1, algorithm='ball_tree', metric='manhattan')

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred, ordered_UNSW_NB15_Label_names)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred,ordered_UNSW_NB15_Label_names))


Training model...
Predicting...

Confusion matrix:
 [[7128    1   38   46    2  122    0    0    1    0]
 [  14 3715   82    2   17   22    1    0    0    0]
 [  76    3 1555  159  377   53    0    1    2    0]
 [ 136    1  167  845   53   27    0    1    1    0]
 [  24    6  378   46  351   14    0    0    0    0]
 [  59    1  111   39   48  426    0    1    2    0]
 [   1    0   55   32   21    0   11    0    0    0]
 [   4    0   65   40    8    0    0    1    0    0]
 [  10    1   19    5    0   30    0    0    1    0]
 [   0    0    4    1    1    3    0    0    0    0]]
Computing performance metrics

Accuracy result: 0.8521892269387259

Classification report:
                precision    recall  f1-score   support

        Normal       0.96      0.97      0.96      7338
       Generic       1.00      0.96      0.98      3853
      Exploits       0.63      0.70      0.66      2226
       Fuzzers       0.70      0.69      0.69      1231
           DoS       0.40      0.43      0.41

/home/will/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [469]:
print_cm(results, short_cat_name, True)

       t/p       Normal   Generic  Exploits   Fuzzers       DoS    Reconn  Analysis  Backdoor Shellcode     Worms 
       Normal    7128.0       1.0      38.0      46.0       2.0     122.0         -         -       1.0         - 
      Generic      14.0    3715.0      82.0       2.0      17.0      22.0       1.0         -         -         - 
     Exploits      76.0       3.0    1555.0     159.0     377.0      53.0         -       1.0       2.0         - 
      Fuzzers     136.0       1.0     167.0     845.0      53.0      27.0         -       1.0       1.0         - 
          DoS      24.0       6.0     378.0      46.0     351.0      14.0         -         -         -         - 
       Reconn      59.0       1.0     111.0      39.0      48.0     426.0         -       1.0       2.0         - 
     Analysis       1.0         -      55.0      32.0      21.0         -      11.0         -         -         - 
     Backdoor       4.0         -      65.0      40.0       8.0         -       